# Nature NIH conflict of interest investigation

Working process to determine the percentage of grants given to each institution that have a FCOI (financial conflict of interest) investigation associated with them. 

`FCOI working doc.xlsx` - NIH FOI document listing all conflict of interest reports that investigators filed in 2012 and 2013

**NIH Research Portfolio Online Reporting Tools (RePORT)** - A repository of NIH-funded research projects and access publications and patents resulting from that funding.

**ExPORTER Data Catalog** [http://exporter.nih.gov/ExPORTER_Catalog.aspx](http://exporter.nih.gov/ExPORTER_Catalog.aspx) - makes downloadable versions of the data accessed through the RePORT Expenditures and Results (RePORTER) interface available to the public. 

## Download and unzip NIH Reporter files relating to fiscal years 2011-14

In [1]:
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2014.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2013.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2012.zip
!curl -O http://exporter.nih.gov/CSVs/final/RePORTER_PRJ_C_FY2011.zip
    
!unzip RePORTER_PRJ_C_FY2014.zip 
!unzip RePORTER_PRJ_C_FY2013.zip
!unzip RePORTER_PRJ_C_FY2012.zip
!unzip RePORTER_PRJ_C_FY2011.zip

!rm RePORTER_PRJ_C_FY2014.zip
!rm RePORTER_PRJ_C_FY2013.zip
!rm RePORTER_PRJ_C_FY2012.zip
!rm RePORTER_PRJ_C_FY2011.zip

We should now have four CSV files relating to 2011,2012,2013 and 2014.

## Clean and combine the NIH reporter documents

Extract just the relevant columns from the four NIH documents into new documents and Convert `iso-8859-1` to `utf-8`.

In [4]:
!in2csv -e iso-8859-1 ref/csv/RePORTER_PRJ_C_FY2014.csv | csvcut -c 'APPLICATION_ID, PROJECT_TITLE, PI_NAMEs, ORG_NAME, FUNDING_MECHANISM, FY, PROJECT_START, PROJECT_END, TOTAL_COST' > nih_2014.csv
!in2csv -e iso-8859-1 ref/csv/RePORTER_PRJ_C_FY2013.csv | csvcut -c 'APPLICATION_ID, PROJECT_TITLE, PI_NAMEs, ORG_NAME, FUNDING_MECHANISM, FY, PROJECT_START, PROJECT_END, TOTAL_COST' > nih_2013.csv
!in2csv -e iso-8859-1 ref/csv/RePORTER_PRJ_C_FY2012.csv | csvcut -c 'APPLICATION_ID, PROJECT_TITLE, PI_NAMEs, ORG_NAME, FUNDING_MECHANISM, FY, PROJECT_START, PROJECT_END, TOTAL_COST' > nih_2012.csv
!in2csv -e iso-8859-1 ref/csv/RePORTER_PRJ_C_FY2011.csv | csvcut -c 'APPLICATION_ID, PROJECT_TITLE, PI_NAMEs, ORG_NAME, FUNDING_MECHANISM, FY, PROJECT_START, PROJECT_END, TOTAL_COST' > nih_2011.csv

Combine all of the entries for all four years into one CSV file:

In [6]:
!csvstack nih_2014.csv nih_2013.csv nih_2012.csv nih_2011.csv > nih_all.csv

In [10]:
!wc -l nih_all.csv #Count the number of lines in the file

  315065 nih_all.csv


In [9]:
!csvcut -n nih_all.csv #Get a numbered list of the columns

  1: APPLICATION_ID
  2: PROJECT_TITLE
  3: PI_NAMEs
  4: ORG_NAME
  5: FUNDING_MECHANISM
  6: FY
  7: PROJECT_START
  8: PROJECT_END
  9: TOTAL_COST


The regulation does not apply to Phase I Small Business Innovation Research (SBIR) and Small Business Technology Transfer (STTR) applications/awards so remove all `SBIR-STTR` entires from the `FUNDING_MECHANISM` column.

In [15]:
!csvgrep -c 5 -r "SBIR-STTR" -i nih_all.csv > temp.csv && mv temp.csv nih_all.csv

In [16]:
!wc -l nih_all.csv #Count the number of lines in the file

  308399 nih_all.csv


##Import pandas, pandasql and csvkit

In [2]:
import pandas
import pandasql
import csvkit

## Load the combined NIH csv into a pandas dataframe

In [16]:
nih_df = pandas.read_csv("./nih_all.csv")
nih_df.rename(columns = lambda x: x.replace(' ', '_').lower(), inplace=True)

nih_org_series = nih_df['org_name'].value_counts()
nih_org_df = pandas.DataFrame(nih_org_series, columns=['total-grants'])

In [38]:
# Each pi_name ends with a semi-colon
# Well need to remove these in order to merge with the FCOI names
print "Before: "
print nih_df['pi_names'].head()

nih_df['pi_names'] = nih_df['pi_names'].map(lambda x: str(x)[:-1])

print "\n"
print "After: "
print nih_df['pi_names'].head()

Before: 
0                                     BAR-SAGI, DAFNA;
1                                     BAR-SAGI, DAFNA;
2    ELLNER, JERROLD J. (contact);JOLOBA, MOSES LUT...
3    DESHPANDE, SMITA N;MANSOUR, HADER A.;NIMGAONKA...
4    TIRSCHWELL, DAVID L;ZUNT, JOSEPH RAYMOND (cont...
Name: pi_names, dtype: object


After: 
0                                      BAR-SAGI, DAFNA
1                                      BAR-SAGI, DAFNA
2    ELLNER, JERROLD J. (contact);JOLOBA, MOSES LUT...
3    DESHPANDE, SMITA N;MANSOUR, HADER A.;NIMGAONKA...
4    TIRSCHWELL, DAVID L;ZUNT, JOSEPH RAYMOND (cont...
Name: pi_names, dtype: object


In [17]:
# Count of grants per institution
nih_org_df.head(3)

,total-grants
JOHNS HOPKINS UNIVERSITY,6705
"UNIVERSITY OF CALIFORNIA, SAN FRANCISCO",6289
UNIVERSITY OF PENNSYLVANIA,5807


In [39]:
nih_person_series = nih_df['pi_names'].value_counts()
nih_person_df = pandas.DataFrame(nih_person_series, columns=['total-grants'])

In [40]:
# Count of grants per person
nih_person_df.head(4)

,total-grants
",",1555
"HEIMBROOK, DAVID",235
"WEINER, GEORGE J",154
"CARBONE, MICHELE",114


## Load the FCOI data into a pandas dataframe

In [7]:
!in2csv --sheet Colored "FCOI working doc.xlsx" > fcoi.csv

In [31]:
!csvcut -n fcoi.csv #Get a numbered list of the columns

  1: FCOI_ID
  2: GRANT NUMBER
  3: FCOI REPORT TYPE
  4: FCOI INVESTIGATOR NAME
  5: FCOI SFI NATURE DESCRIPTION
  6: GRANT TITLE
  7: CONTACT PD/PI NAME
  8: SUBMITTING INSTITUTION
  9: SUBRECIPIENT INSTITUTION NAME
 10: FCOI AMOUNT VALUE
 11: Value ordered


In [25]:
fcoi_df = pandas.read_csv("./fcoi.csv")
fcoi_df.rename(columns = lambda x: x.replace(' ','_').lower(),  inplace=True)

fcoi_org_series = fcoi_df['submitting_institution'].value_counts()
fcoi_org_df = pandas.DataFrame(fcoi_org_series, columns=['total-fcoi-claims'])

fcoi_person_series = fcoi_df['contact_pd/pi_name'].value_counts()
fcoi_person_df = pandas.DataFrame(fcoi_person_series, columns=['totas-fcoi-claims'])

In [23]:
# Count of total fcoi claims per institution
fcoi_org_df.head(3)

,total-fcoi-claims
UNIVERSITY OF WISCONSIN-MADISON,657
UNIVERSITY OF ALABAMA AT BIRMINGHAM,522
"BRIGHAM AND WOMEN'S HOSP., INC.",172


In [26]:
# Count of total fcoi claims per person
fcoi_person_df.head(3)

,totas-fcoi-claims
"LUO, MING",70
"Whitley, Richard J.",62
"ROBB, RICHARD A",53


## Join the NIH and FCOI data frames for comparison

In [30]:
# pd.merge(left_frame, right_frame, on='key', how='inner')
org_merge = pandas.merge(fcoi_org_df, nih_org_df, left_index=True, right_index=True, how='inner')

org_merge.to_csv("./institution_fcoi_nih.csv")

org_merge.head()

,total-fcoi-claims,total-grants
JOHNS HOPKINS UNIVERSITY,66,6705
"UNIVERSITY OF CALIFORNIA, SAN FRANCISCO",29,6289
UNIVERSITY OF PENNSYLVANIA,94,5807
UNIVERSITY OF MICHIGAN,76,5336
UNIVERSITY OF WASHINGTON,46,5146


In [41]:
# Full outer join as there are lots of missmatches
person_merge = pandas.merge(fcoi_person_df, nih_person_df, left_index=True, right_index=True, how='outer')

person_merge.to_csv("./person_fcoi_nih.csv")

person_merge.head()

,totas-fcoi-claims,total-grants
",",NaN,1555
"AAGAARD-TILLERY, KJERSTI MARIE",NaN,7
"AAKALU, VINAY",NaN,1
"AALBERSBERG, WILLIAM",NaN,1
"AALBERTS, DANIEL PAUL",NaN,1


##Sum up the types of claims per institution

In [42]:
fcoi_type_of_claim_list = fcoi_df['fcoi_sfi_nature_description'].unique().tolist()

len(fcoi_type_of_claim_list)

# for i, v in enumerate(fcoi_type_of_claim_list):
#     fcoi_org_df[v] = 0

# # print fcoi_org_df[0:1]['Payment for services (e.g., consulting fees, honoraria, paid authorship)']
# print fcoi_org_df.loc['UNIVERSITY OF WISCONSIN-MADISON']['Payment for services (e.g., consulting fees, honoraria, paid authorship)']




130

In [106]:
reader = csvkit.DictReader(open('./fcoi.csv', 'rU'), delimiter=',')

for row in reader:
#     list_all_orgs.append(row)
    myOrg = row['SUBMITTING INSTITUTION']
    myType = row['FCOI SFI NATURE DESCRIPTION']
    fcoi_org_df.loc[myOrg][myType] += 1
    
print fcoi_org_df.loc['UNIVERSITY OF WISCONSIN-MADISON']

fcoi_org_df.to_csv("./fcoi_org.csv")

total                                                           657
Payment for services (e.g., consulting fees, honoraria, paid authorship)    272
Equity Interest -  Non-publicly traded entity ( e.g., stock, stock option, or other ownership interest)    324
IP rights                                                         0
Founder with equity interest                                      0
Reimbursed or sponsored travel                                    1
Intellectual property rights (e.g., royalties, patents, copyrights) not from the awardee Institution      1
CEO/Founder                                                       0
Equity Interest - Publicly traded entity (e.g., stock, stock option, or other ownership interest)     33
Fiduciary Board of Directors                                      0
Member, Advisory Board                                            0
Licensed IP through Duke                                          0
Management Role                                     

In [ ]:
fcoi_person_df = fcoi_df['contact_pd/pi_name'].value_counts()

fcoi_orgs = fcoi_df['submitting_institution'].unique()

# fcoi_orgs_df= pandas.DataFrame(fcoi_orgs)

# print fcoi_org_df.head()






# dict_summed_orgs = {}

# for i, v in enumerate(fcoi_orgs_list):
#     dict_summed_orgs[v] = {}
#     for y, x in enumerate(fcoi_type_of_claim_list):
#         dict_summed_orgs[v][x] = 0
    
# print dict_summed_orgs['UNIV OF NORTH CAROLINA CHAPEL HILL']

# q = """
# SELECT
#     submitting_institution, fcoi_sfi_nature_description
# FROM
#     fcoi_df
# GROUP BY
#     submitting_institution
# """

# type_per_org_df = pandasql.sqldf(q.lower(), locals())

# type_per_org_df.to_csv("./group_test.csv")

# grouped = fcoi_df.groupby('submitting_institution')



# for org, y in grouped:
#     print y[0:1]['fcoi_sfi_nature_description']

# print type_per_org_df.head()

# type_per_org_df.to_csv("./test-type.csv")